### 1. Setup & CSV einlesen


In [1]:
# -----------------------------------------------------------
# 1) Import necessary libraries
# -----------------------------------------------------------
import pandas as pd
import numpy as np
import re
import sys
!{sys.executable} -m pip install sqlalchemy pymysql cryptography

# Pretty display options for DataFrames
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)

# -----------------------------------------------------------
# 2) Load the dataset (adjust filename if needed)
# -----------------------------------------------------------
df = pd.read_csv("schweizer_wanderwege_alle.csv")

# First quick look
df.head()


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\rexhe\DA\Exercises\.venv\Scripts\python.exe -m pip install --upgrade pip


,web_scraper_order,web_scraper_start_url,detail_url,title,location,difficulty_level,duration,distance,physical_demand,ascent,descent,target_group,season,recommended_period
0,1766396370-1,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Husky-Traum,Muotathal,NaN,NaN,4m,hoch,NaN,NaN,"Zyklus 1 (Grundschule / 6 - 9 Jahre alt), Zykl...",ganzjährig,NaN
1,1766396372-2,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Aufregende Ausblicke auf dem Hochalp Trail,"Urnäsch, Anker\n ...",NaN,6h 45min,11.4km,hoch,720m,720m,NaN,Winter,Dezember - März
2,1766396375-3,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Den ganzen Tag im Schnee,Atzmännig SSA\n ...,NaN,1h 40min,4.2km,mittel,140m,510m,für Familien,Winter,Dezember - März
3,1766396377-4,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Panorama Trail in Obwalden,Langis\n ...,NaN,4h 30min,12.3km,hoch,565m,565m,NaN,Winter,Dezember - März
4,1766396379-5,https://www.schweizer-wanderwege.ch/de/wanderv...,https://www.schweizer-wanderwege.ch/de/wanderv...,Schneeschuhwanderung auf den Chasseron,"Les Rasses, village\n ...",NaN,4h 20min,10km,hoch,450m,450m,NaN,Winter,Dezember - März


### 2. Remove unnecessary scraper columns

In [2]:
# -----------------------------------------------------------
# Remove columns created automatically by the web-scraper
# These do not contain useful analytical information
# -----------------------------------------------------------
columns_to_drop = ["web_scraper_order", "web_scraper_start_url"]

df = df.drop(columns=columns_to_drop, errors="ignore")

# Rename the "detail_url" column to a simpler name
df = df.rename(columns={"detail_url": "url"})

df.head()


,url,title,location,difficulty_level,duration,distance,physical_demand,ascent,descent,target_group,season,recommended_period
0,https://www.schweizer-wanderwege.ch/de/wanderv...,Husky-Traum,Muotathal,NaN,NaN,4m,hoch,NaN,NaN,"Zyklus 1 (Grundschule / 6 - 9 Jahre alt), Zykl...",ganzjährig,NaN
1,https://www.schweizer-wanderwege.ch/de/wanderv...,Aufregende Ausblicke auf dem Hochalp Trail,"Urnäsch, Anker\n ...",NaN,6h 45min,11.4km,hoch,720m,720m,NaN,Winter,Dezember - März
2,https://www.schweizer-wanderwege.ch/de/wanderv...,Den ganzen Tag im Schnee,Atzmännig SSA\n ...,NaN,1h 40min,4.2km,mittel,140m,510m,für Familien,Winter,Dezember - März
3,https://www.schweizer-wanderwege.ch/de/wanderv...,Panorama Trail in Obwalden,Langis\n ...,NaN,4h 30min,12.3km,hoch,565m,565m,NaN,Winter,Dezember - März
4,https://www.schweizer-wanderwege.ch/de/wanderv...,Schneeschuhwanderung auf den Chasseron,"Les Rasses, village\n ...",NaN,4h 20min,10km,hoch,450m,450m,NaN,Winter,Dezember - März


### 3. Remove duplicate routes using the unique URL

In [3]:
# -----------------------------------------------------------
# Remove duplicate hiking routes based on the URL.
# The URL uniquely identifies a hiking detail page.
# -----------------------------------------------------------
before = len(df)
df = df.drop_duplicates(subset=["url"])
after = len(df)

print("Rows before removing duplicates:", before)
print("Rows after:", after)


Rows before removing duplicates: 375
Rows after: 375


### 4. Convert distance to a numeric float (km)

In [4]:
# -----------------------------------------------------------
# Convert values like "11km" into the numeric value 11.0
# -----------------------------------------------------------

def parse_distance_km(x):
    """
    Convert strings such as '11km' or '11 km' to a float.
    If the value cannot be parsed, return NaN.
    """
    if pd.isna(x):
        return np.nan

    m = re.match(r"([\d.]+)\s*(km|m)$", str(x).lower())
    if not m:
        return np.nan

    value, unit = m.groups()
    return float(value) / 1000 if unit == "m" else float(value)

df["distance_km"] = df["distance"].apply(parse_distance_km)

df[["distance", "distance_km"]].head()


,distance,distance_km
0,4m,0.004
1,11.4km,11.400
2,4.2km,4.200
3,12.3km,12.300
4,10km,10.000


### 5. Convert duration into total minutes

In [5]:
# -----------------------------------------------------------
# Convert strings like "3h 15min" into total minutes.
# Example: "3h 15min" → 195 minutes
#          "2h" → 120
#          "45min" → 45
# -----------------------------------------------------------

def duration_to_minutes(x):
    if pd.isna(x):
        return np.nan
    
    text = str(x).lower().replace(" ", "")
    
    # Extract hours and minutes
    hours = 0
    minutes = 0

    h_match = re.search(r"(\d+)h", text)
    m_match = re.search(r"(\d+)min", text)

    if h_match:
        hours = int(h_match.group(1))
    if m_match:
        minutes = int(m_match.group(1))

    total = hours * 60 + minutes
    return total if total > 0 else np.nan

df["duration_min"] = df["duration"].apply(duration_to_minutes)

df[["duration", "duration_min"]].head()


,duration,duration_min
0,NaN,NaN
1,6h 45min,405.0
2,1h 40min,100.0
3,4h 30min,270.0
4,4h 20min,260.0


### 6. Convert ascent & descent into integer meters

In [6]:
# -----------------------------------------------------------
# Convert values like "310m" into integer values such as 310
# -----------------------------------------------------------

def height_to_m(x):
    if pd.isna(x):
        return np.nan
    match = re.findall(r"\d+", str(x))
    return int(match[0]) if match else np.nan

df["ascent_m"]  = df["ascent"].apply(height_to_m)
df["descent_m"] = df["descent"].apply(height_to_m)

df[["ascent", "ascent_m", "descent", "descent_m"]].head()


,ascent,ascent_m,descent,descent_m
0,NaN,NaN,NaN,NaN
1,720m,720.0,720m,720.0
2,140m,140.0,510m,510.0
3,565m,565.0,565m,565.0
4,450m,450.0,450m,450.0


### 7. Split the location field into area, region, and canton

In [7]:
# -----------------------------------------------------------
# The 'location' field sometimes looks like:
# "Abländschen, Mittelberg — Saanen • BE"
#
# We split this into:
# - area        (left side of —)
# - region      (right side of —)
# - canton      (after the bullet •)
# -----------------------------------------------------------

def extract_canton(loc):
    if pd.isna(loc): return np.nan
    # Entferne Zeilenumbrüche und überflüssige Whitespaces
    loc_clean = ' '.join(str(loc).split())
    parts = loc_clean.split("•")
    return parts[-1].strip() if len(parts) > 1 else np.nan

def extract_region(loc):
    if pd.isna(loc): return np.nan
    loc_clean = ' '.join(str(loc).split())
    left = loc_clean.split("•")[0]
    if "—" in left:
        return left.split("—")[1].strip()
    return left.strip()

def extract_area(loc):
    if pd.isna(loc): return np.nan
    loc_clean = ' '.join(str(loc).split())
    left = loc_clean.split("•")[0]
    if "—" in left:
        return left.split("—")[0].strip()
    return left.strip()

df["canton"] = df["location"].apply(extract_canton)
df["region"] = df["location"].apply(extract_region)

df["area"]   = df["location"].apply(extract_area)

df[["location", "area", "region", "canton"]].head()

,location,area,region,canton
0,Muotathal,Muotathal,Muotathal,NaN
1,"Urnäsch, Anker\n ...","Urnäsch, Anker","Urnäsch, Anker",AR
2,Atzmännig SSA\n ...,Atzmännig SSA,"Atzmännig, Schutt",SG
3,Langis\n ...,Langis,Langis,OW
4,"Les Rasses, village\n ...","Les Rasses, village","Les Rasses, village",VD


### 8. Encode difficulty & physical demand (categorical → numeric)

In [8]:
# -----------------------------------------------------------
# Convert categorical difficulty levels to numeric values
# Example: T1→1, T2→2, ..., T6→6
# WICHTIG: difficulty_level bleibt mit T-Präfix (z.B. "T2")
# -----------------------------------------------------------

difficulty_map = {
    "T1": 1, "T2": 2, "T3": 3,
    "T4": 4, "T5": 5, "T6": 6
}

# Normalisiere nur Whitespace, entferne NICHT das T
df["difficulty_level"] = df["difficulty_level"].astype(str).str.strip().str.replace(r'\s+', '', regex=True)
df["difficulty_num"] = df["difficulty_level"].map(difficulty_map)


df[["difficulty_level", "difficulty_num"]].head()

,difficulty_level,difficulty_num
0,nan,NaN
1,nan,NaN
2,nan,NaN
3,nan,NaN
4,nan,NaN


In [9]:
# -----------------------------------------------------------
# Convert "leicht/mittel/hoch" to numeric 1/2/3
# -----------------------------------------------------------

physical_demand_map = {"leicht":1, "mittel":2, "hoch":3}

df["physical_demand"] = df["physical_demand"].astype(str).str.lower().str.strip()
df["physical_demand_num"] = df["physical_demand"].map(physical_demand_map)

df[["physical_demand", "physical_demand_num"]].head()


,physical_demand,physical_demand_num
0,hoch,3.0
1,hoch,3.0
2,mittel,2.0
3,hoch,3.0
4,hoch,3.0


### 9. Open-Data-Enrichment with longitude & latitude

In [10]:
# -----------------------------------------------------------
# Geocode Swiss hiking locations using OpenStreetMap (Nominatim)
# Returns latitude and longitude for each unique area
# -----------------------------------------------------------

import requests
import time

RATE_LIMIT_SECONDS = 0.25

def geocode_osm(place):
    """
    Query OpenStreetMap Nominatim API to get latitude and longitude
    for a given place in Switzerland.
    Returns (lat, lon) or (None, None) if nothing is found.
    """
    if pd.isna(place):
        return None, None

    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{place}",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, params=params, headers=headers)
        print(f"API Request: {response.url}")
        data = response.json()

        if len(data) == 0:
            return None, None

        lat = float(data[0]["lat"])
        lon = float(data[0]["lon"])
        return lat, lon

    except:
        return None, None
    finally:
        # Avoid hitting rate limits
        time.sleep(RATE_LIMIT_SECONDS)


def load_geocoding_cache():
    """Load existing geocoding cache from CSV file."""
    try:
        cache_df = pd.read_csv("geocoding_cache.csv")
        cache_dict = dict(zip(cache_df["url"], zip(cache_df["latitude"], cache_df["longitude"])))
        print(f"Loaded {len(cache_dict)} cached coordinates")
        return cache_dict
    except FileNotFoundError:
        print("No cache file found, creating new one")
        return {}


def initialize_cache_file(file_handle):
    """Write CSV header if cache file is empty."""
    if file_handle.tell() == 0:
        file_handle.write("url,latitude,longitude\n")
        file_handle.flush()


def get_coordinates(place, identifier, cache_dict):
    """
    Get coordinates for a place, either from cache or via API.
    Returns (lat, lon) tuple.
    """
    if identifier in cache_dict:
        lat, lon = cache_dict[identifier]
        print(f"Found in cache: {place} -> {lat}, {lon}")
        return lat, lon

    lat, lon = geocode_osm(place)

    if lat is None or lon is None:
        lat, lon = np.nan, np.nan
        print(f"Not found: {place} -> {lat}, {lon}")
    else:
        print(f"Geocoded: {place} -> {lat}, {lon}")

    return lat, lon


def save_to_cache(file_handle, identifier, lat, lon):
    """Save coordinates to cache file."""
    file_handle.write(f"{identifier},{lat},{lon}\n")
    file_handle.flush()


def geocode_dataframe(df):
    """
    Add latitude and longitude columns to dataframe by geocoding the 'area' field.
    Uses caching to avoid redundant API calls.
    """
    df["latitude"] = None
    df["longitude"] = None

    cache_dict = load_geocoding_cache()

    with open("geocoding_cache.csv", "a", encoding="utf-8") as f:
        initialize_cache_file(f)

        for idx, row in df.iterrows():
            place = row["area"]
            identifier = row["url"]

            lat, lon = get_coordinates(place, identifier, cache_dict)

            # Save to cache if newly geocoded
            if identifier not in cache_dict:
                save_to_cache(f, identifier, lat, lon)

            df.at[idx, "latitude"] = lat
            df.at[idx, "longitude"] = lon

    return df


# -----------------------------------------------------------
# Apply geocoding
# -----------------------------------------------------------
df = geocode_dataframe(df)

Loaded 375 cached coordinates
Found in cache: Muotathal -> nan, nan
Found in cache: Urnäsch, Anker -> nan, nan
Found in cache: Atzmännig SSA -> nan, nan
Found in cache: Langis -> nan, nan
Found in cache: Les Rasses, village -> nan, nan
Found in cache: Veltheim AG, Bären -> 47.4369232, 8.1474092
Found in cache: Bibern SH, Dorf -> 47.7722392, 8.6785329
Found in cache: Spreitenbach -> nan, nan
Found in cache: Anzère, télécabine -> nan, nan
Found in cache: Les Paccots, Les Rosalys -> nan, nan
Found in cache: Zweisimmen, Sparenmoos -> nan, nan
Found in cache: Hoch-Ybrig, Talst. Laucheren -> nan, nan
Found in cache: Obersaxen, Meierhof Post -> nan, nan
Found in cache: Zeihen, Stauftel -> 47.477424, 8.0897368
Found in cache: Messen, Dorfplatz -> 47.0918287, 7.4475343
Found in cache: Lamone-Cadempino -> 46.0394453, 8.9322308
Found in cache: Gruyères -> 49.7106979, 4.6041316
Found in cache: Miécourt -> 47.4270918, 7.1747627
Found in cache: Huttwil -> 47.1147419, 7.8455436
Found in cache: Malans

### 10. Save the cleaned dataset

In [11]:
# -----------------------------------------------------------
# Export the cleaned dataset so it can be used in the next steps
# (EDA, modelling, database storage, etc.)
# -----------------------------------------------------------

# Convert duration_min, ascent_m, and descent_m to integer for cleaner output
# Only convert if the value is not NaN
df["duration_min"] = df["duration_min"].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df["ascent_m"] = df["ascent_m"].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df["descent_m"] = df["descent_m"].apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Bereinige location: entferne Zeilenumbrüche und überflüssige Whitespaces
df["location_clean"] = df["location"].apply(lambda x: ' '.join(str(x).split()) if pd.notna(x) else x)

# Extrahiere Canton aus der bereinigten Location (nach dem • Symbol)
def extract_canton_clean(loc):
    if pd.isna(loc): return np.nan
    if "•" in str(loc):
        return str(loc).split("•")[-1].strip()
    return np.nan

df["canton_clean"] = df["location_clean"].apply(extract_canton_clean)

# Entferne "• Canton" aus location, behalte nur die Ortsnamen
def clean_location_remove_canton(loc):
    if pd.isna(loc): return loc
    if "•" in str(loc):
        return str(loc).split("•")[0].strip()
    return str(loc).strip()

df["location_clean"] = df["location_clean"].apply(clean_location_remove_canton)

# Wähle nur die gewünschten Spalten aus
columns_to_export = [
    "url",
    "title", 
    "location_clean",
    "canton_clean",
    "difficulty_level",
    "difficulty_num",
    "duration_min",
    "distance_km",
    "ascent_m",
    "descent_m",
    "physical_demand",
    "latitude",
    "longitude"
]

df_export = df[columns_to_export]

# Benenne Spalten für sauberes CSV
df_export = df_export.rename(columns={
    "location_clean": "location",
    "canton_clean": "canton"
})

print(f"✅ Prepared {len(df_export)} rows with {len(columns_to_export)} columns for MySQL export")


✅ Prepared 375 rows with 13 columns for MySQL export


### 11. Store data in MySQL database

In [12]:
# -----------------------------------------------------------
# MySQL connection setup
# Make sure Docker container is running: docker compose up -d
# -----------------------------------------------------------
from sqlalchemy import create_engine

# Database connection string
DB_USER = "root"
DB_PASSWORD = "password"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "wanderwege_db"

# Create database engine
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

# Test connection
try:
    with engine.connect() as conn:
        print("✅ Successfully connected to MySQL database!")
        print(f"📊 Database: {DB_NAME}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    

✅ Successfully connected to MySQL database!
📊 Database: wanderwege_db


In [13]:
# -----------------------------------------------------------
# Write cleaned and enriched data to MySQL
# -----------------------------------------------------------

try:
    # Write DataFrame to MySQL table
    df_export.to_sql(
        name='wanderwege',           # Table name in MySQL
        con=engine,                   # SQLAlchemy engine
        if_exists='replace',          # Drop table if exists and recreate
        index=False,                  # Don't write DataFrame index
        chunksize=1000,              # Write in batches of 1000 rows
        method='multi'                # Use multi-row INSERT for better performance
    )
    
    print(f"✅ Successfully stored {len(df_export)} wanderwege in MySQL!")



    
    # Verify data was written correctly
    row_count_query = "SELECT COUNT(*) as count FROM wanderwege"
    result = pd.read_sql(row_count_query, con=engine)
    print(f"\n✓ Verification: {result['count'][0]} rows in database")
    
except Exception as e:
    print(f"❌ Error writing to database: {e}")


✅ Successfully stored 375 wanderwege in MySQL!

✓ Verification: 375 rows in database
